In [1]:
import os
import pathlib
import numpy as np
import keras
import random
import tensorflow
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization, Input, Add, ReLU, AveragePooling2D, GlobalAveragePooling2D
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

from keras.applications.resnet import ResNet50
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

path = 'C:\\Users\\pc\\Desktop\\GP\\256_ObjectCategories\\256_ObjectCategories\\'
img_size = (64, 64)
input_shape = (64, 64, 3)
num_classes = 257

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
def process(image,label):
    image = tensorflow.cast(image/255. ,tensorflow.float32)
    return image,label

In [3]:
dataDirectory = pathlib.Path(path)

train_data_set = tensorflow.keras.preprocessing.image_dataset_from_directory(
    dataDirectory,
    labels = 'inferred',
    label_mode = 'categorical',
    validation_split = 0.2,
    subset = 'training',
    seed = 1234,
    image_size = img_size,
    batch_size = 128
)
train_data_set = train_data_set.map(process)

val_data_set = tensorflow.keras.preprocessing.image_dataset_from_directory(
    dataDirectory,
    labels = 'inferred',
    label_mode = 'categorical',
    validation_split = 0.2,
    subset = 'validation',
    seed = 1234,
    image_size = img_size,
    batch_size = 128
)

val_data_set = val_data_set.map(process)
print(train_data_set)
AUTOTUNE = tensorflow.data.experimental.AUTOTUNE
train_data_set = train_data_set.cache().prefetch(buffer_size = AUTOTUNE)
val_data_set = val_data_set.cache().prefetch(buffer_size = AUTOTUNE)

Found 30607 files belonging to 257 classes.
Using 24486 files for training.
Found 30607 files belonging to 257 classes.
Using 6121 files for validation.
<MapDataset shapes: ((None, 64, 64, 3), (None, 257)), types: (tf.float32, tf.float32)>


In [4]:
def residual_block(X_start, filters, name, reduce=False, res_conv2d=False):

    nb_filters_1, nb_filters_2, nb_filters_3 = filters
    strides_1 = [2,2] if reduce else [1,1]
        
    X = Conv2D(filters=nb_filters_1, kernel_size=[1,1], strides=strides_1, padding='same', name=name)(X_start)
    X = BatchNormalization()(X)      # default axis-1 is ok
    X = Activation('relu')(X)
    
    X = Conv2D(filters=nb_filters_2, kernel_size=[3,3], strides=[1,1], padding='same')(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    
    X = Conv2D(filters=nb_filters_3, kernel_size=[1,1], strides=[1,1], padding='same')(X)
    X = BatchNormalization()(X)
    
    if res_conv2d:
        X_res = Conv2D(filters=nb_filters_3, kernel_size=[1,1], strides=strides_1, padding='same')(X_start)
        X_res = BatchNormalization()(X_res)
    else:
        X_res = X_start
        
    X = Add()([X, X_res])
    X = Activation('relu')(X)
    return X

In [5]:
def resnet50(nb_classes):
    
    X_input = Input(shape = input_shape)

    # conv1
    X = Conv2D(filters=64, kernel_size=[7,7], strides=[2,2], padding='same', name='conv1')(X_input)
    X = BatchNormalization(name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D([3,3], strides=[2,2])(X)

    # conv2_x
    X = residual_block(X, filters=[64, 64, 256], name='conv2_a', reduce=False, res_conv2d=True)
    X = residual_block(X, filters=[64, 64, 256], name='conv2_b')
    X = residual_block(X, filters=[64, 64, 256], name='conv2_c')

    # conv3_x
    X = residual_block(X, filters=[128, 128, 512], name='conv3_a', reduce=True, res_conv2d=True)
    X = residual_block(X, filters=[128, 128, 512], name='conv3_b')
    X = residual_block(X, filters=[128, 128, 512], name='conv3_c')
    X = residual_block(X, filters=[128, 128, 512], name='conv3_d')

    # conv4_x
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_a', reduce=True, res_conv2d=True)
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_b')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_c')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_d')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_e')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_f')

    # conv5_x
    X = residual_block(X, filters=[512, 512, 2048], name='conv5_a', reduce=True, res_conv2d=True)
    X = residual_block(X, filters=[512, 512, 2048], name='conv5_b')
    X = residual_block(X, filters=[512, 512, 2048], name='conv5_c')

    X = GlobalAveragePooling2D(name='avg_pool')(X)
    X = Flatten()(X)
    X = Dense(units=nb_classes, activation='softmax')(X)
    
    model = Model(inputs = X_input, outputs = X)
    return model

In [6]:
model = resnet50(nb_classes = num_classes)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 32, 32, 64)   9472        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 32, 32, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 64)   0           bn_conv1[0][0]                   
______________________________________________________________________________________________

In [48]:
##  시험용 
model = ResNet50(
    include_top = False,
    weights = None,
    input_shape = input_shape,
    pooling = 'max'
    )

x = model.output
x = Dense(1024, name = 'fully', activation = 'relu')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dense(512)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dense(num_classes, activation = 'softmax', name = 'softmax')(x)
model = Model(inputs = model.input, outputs = x)
model.summary()

Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 70, 70, 3)    0           input_6[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 32, 32, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 32, 32, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [7]:
model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy'])

In [8]:
model.fit(train_data_set, validation_data = val_data_set, epochs = 20)

Epoch 1/20
  3/192 [..............................] - ETA: 24:39 - loss: 7.5533 - accuracy: 0.0148     

KeyboardInterrupt: 